# 📊 DEEPCHECKS CRIME LA - NIVEAU 1 : INTÉGRITÉ DES DONNÉES

## 🎯 Objectif
Vérifier l'intégrité des données de criminalité de Los Angeles avant l'entraînement :
- Détection des valeurs manquantes critiques
- Identification des doublons
- Vérification de la cohérence des features
- Analyse des outliers géographiques et temporels

---

## 📦 Imports et Configuration

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Ajouter src au path pour imports (si besoin)
sys.path.append(os.path.join(os.getcwd(), '..'))

print("✅ Imports de base effectués")

# Deepchecks Tabular
try:
    from deepchecks.tabular import Dataset
    from deepchecks.tabular.suites import data_integrity
    DEEPCHECKS_AVAILABLE = True
    print("✅ Deepchecks importé")
except ImportError as e:
    print(f"⚠️ Deepchecks non disponible: {e}")
    print("Installation: pip install deepchecks[all]")
    DEEPCHECKS_AVAILABLE = False

# Configuration des chemins
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
REPORTS_DIR = BASE_DIR / 'reports' / 'deepchecks'
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print("="*80)
print("📊 DEEPCHECKS CRIME LA - NIVEAU 1 : INTÉGRITÉ DES DONNÉES (DONNÉES BRUTES UNIQUEMENT)")
print("="*80)
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📁 Base: {BASE_DIR}")
print(f"📁 Données brutes: {RAW_DATA_DIR}")
print(f"📁 Reports: {REPORTS_DIR}")
print()

def load_crime_raw_data():
    print("📦 Chargement exclusif des données BRUTES crime")
    print("-" * 80)
    
    raw_files = list(RAW_DATA_DIR.glob('*.csv'))
    
    if not raw_files:
        raise FileNotFoundError(f"❌ Aucun fichier CSV trouvé dans {RAW_DATA_DIR}")
    
    data_file = raw_files[0]
    print(f"✅ Données brutes chargées: {data_file.name}")
    df = pd.read_csv(data_file)
    
    print(f"   Shape: {df.shape}")
    print(f"   Colonnes: {len(df.columns)}")
    if 'DATE OCC' in df.columns:
        print(f"   Période: {df['DATE OCC'].min()} à {df['DATE OCC'].max()}")
    print()
    
    return df

df_crime = load_crime_raw_data()

print("📋 Aperçu des données brutes:")
display(df_crime.head())

print(f"\n📊 Info des colonnes:")
df_crime.info()

def prepare_crime_dataset_for_deepchecks(df):
    print("📝 Préparation Dataset Deepchecks")
    print("-" * 80)
    
    label_col = 'Crm Cd Desc' if 'Crm Cd Desc' in df.columns else None
    
    feature_cols = [col for col in ['LAT', 'LON', 'Vict Age', 'AREA'] if col in df.columns]
    
    cat_features = [col for col in df.select_dtypes(include=['object']).columns if col != label_col]
    
    print(f"✅ Configuration Dataset:")
    print(f"   Label: {label_col}")
    print(f"   Features numériques détectées: {feature_cols}")
    print(f"   Features catégorielles (exemple): {cat_features[:5]}{'...' if len(cat_features) > 5 else ''}")
    
    if DEEPCHECKS_AVAILABLE and label_col:
        dataset = Dataset(df, label=label_col, cat_features=cat_features)
        return dataset, label_col, feature_cols
    else:
        return None, label_col, feature_cols

crime_dataset, label_column, feature_columns = prepare_crime_dataset_for_deepchecks(df_crime)

def run_crime_data_integrity_checks(dataset, df):
    print("\n" + "="*80)
    print("📊 NIVEAU 1: INTÉGRITÉ DES DONNÉES BRUTES CRIME LA")
    print("="*80)
    
    results = {}
    
    if DEEPCHECKS_AVAILABLE and dataset:
        print("\n🔍 Exécution des checks Deepchecks...")
        integrity_suite = data_integrity()
        result = integrity_suite.run(dataset)
        
        # 1. Affichage DIRECT dans le notebook → TOUJOURS parfait et interactif !
        print("\n📊 AFFICHAGE DU RAPPORT DIRECTEMENT DANS LE NOTEBOOK :\n")
        result.show()
        
        # 2. Sauvegarde HTML en mode widget (interactif, fonctionne avec anywidget installé)
        integrity_report_path = REPORTS_DIR / 'crime_raw_data_integrity_report.html'
        result.save_as_html(str(integrity_report_path))  # as_widget=True par défaut
        
        print(f"\n✅ Rapport HTML sauvegardé : {integrity_report_path.name}")
        print("   Ouvre-le dans ton navigateur si tu veux une version fichier séparée.")
        
        results['deepchecks_result'] = result
    
    # Analyses manuelles (inchangées)
    if {'LAT', 'LON'}.issubset(df.columns):
        lat_valid = ((df['LAT'] >= 33.0) & (df['LAT'] <= 35.0)).sum()
        lon_valid = ((df['LON'] >= -119.0) & (df['LON'] <= -117.0)).sum()
        results['gps_valid_pct'] = min(lat_valid, lon_valid) / len(df) * 100
    
    missing_analysis = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing_analysis / len(df) * 100).round(2)
    results['missing_analysis'] = missing_analysis
    results['missing_pct'] = missing_pct
    
    if label_column:
        crime_dist = df[label_column].value_counts()
        results['crime_types_count'] = len(crime_dist)
        results['most_common_crime'] = crime_dist.index[0]
    
    return results

integrity_results = run_crime_data_integrity_checks(crime_dataset, df_crime)

# === Le reste du code (vérifications critiques, résumé, etc.) reste identique ===
# (je le copie pour que ce soit complet)

def critical_crime_data_checks(df):
    print("\n🚨 VÉRIFICATIONS CRITIQUES (DONNÉES BRUTES)")
    print("-" * 80)
    
    issues = []
    
    if {'LAT', 'LON'}.issubset(df.columns):
        invalid_coords = ((df['LAT'] < 33.0) | (df['LAT'] > 35.0) | 
                          (df['LON'] < -119.0) | (df['LON'] > -117.0) | 
                          df['LAT'].isna() | df['LON'].isna()).sum()
        pct = invalid_coords / len(df) * 100
        if pct > 10:
            issues.append(f"🔴 CRITIQUE: {invalid_coords} coordonnées GPS invalides ({pct:.1f}%)")
        else:
            print(f"✅ Coordonnées GPS: {invalid_coords} invalides ({pct:.1f}%) - OK")
    
    if 'DATE OCC' in df.columns:
        df_date = df.copy()
        df_date['DATE OCC'] = pd.to_datetime(df_date['DATE OCC'], errors='coerce')
        invalid_dates = df_date['DATE OCC'].isna().sum()
        pct = invalid_dates / len(df) * 100
        if pct > 5:
            issues.append(f"🔴 CRITIQUE: {invalid_dates} dates invalides ({pct:.1f}%)")
        else:
            print(f"✅ Dates: {invalid_dates} invalides ({pct:.1f}%) - OK")
    
    if 'Vict Age' in df.columns:
        invalid_ages = ((df['Vict Age'] < 0) | (df['Vict Age'] > 120)).sum()
        if invalid_ages > 0:
            issues.append(f"🟡 ATTENTION: {invalid_ages} âges aberrants")
        else:
            print(f"✅ Âges victimes: OK")
    
    duplicates = df.duplicated().sum()
    pct_dup = duplicates / len(df) * 100
    if pct_dup > 1:
        issues.append(f"🔴 CRITIQUE: {duplicates} doublons ({pct_dup:.1f}%)")
    else:
        print(f"✅ Doublons: {duplicates} ({pct_dup:.1f}%) - Acceptable")
    
    if issues:
        print(f"\n⚠️ {len(issues)} PROBLÈME(S) CRITIQUE(S) :")
        for issue in issues:
            print(f"   {issue}")
        return False
    else:
        print(f"\n✅ TOUTES LES VÉRIFICATIONS CRITIQUES PASSÉES")
        return True

data_quality_ok = critical_crime_data_checks(df_crime)

print("\n" + "="*80)
print("✅ NIVEAU 1 : INTÉGRITÉ DES DONNÉES BRUTES - TERMINÉ")
print("="*80)

print(f"\n📊 Résumé Global (données brutes):")
print(f"   Lignes: {df_crime.shape[0]:,} | Colonnes: {df_crime.shape[1]}")
print(f"   Qualité globale: {'✅ BONNE' if data_quality_ok else '⚠️ PROBLÈMES À CORRIGER'}")

if 'gps_valid_pct' in integrity_results:
    print(f"   GPS valides estimés: {integrity_results['gps_valid_pct']:.1f}%")
if 'crime_types_count' in integrity_results:
    print(f"   Types de crimes distincts: {integrity_results['crime_types_count']}")
    print(f"   Crime le plus fréquent: {integrity_results['most_common_crime']}")

print(f"\n💡 Prochaines étapes:")
if data_quality_ok:
    print("   1. ✅ Données brutes OK → passe au préprocessing")
else:
    print("   1. 🔧 Nettoie surtout les 41 âges aberrants (valeurs négatives ou >120)")

print(f"\n📋 Rapport interactif visible ci-dessus dans le notebook !")
print(f"   Fichier HTML aussi disponible ici : {REPORTS_DIR / 'crime_raw_data_integrity_report.html'}")

print("\n" + "="*80)

✅ Imports de base effectués
✅ Deepchecks importé
📊 DEEPCHECKS CRIME LA - NIVEAU 1 : INTÉGRITÉ DES DONNÉES (DONNÉES BRUTES UNIQUEMENT)
📅 Date: 2025-12-21 18:33:59
📁 Base: d:\Ingénierie3\MLops
📁 Données brutes: d:\Ingénierie3\MLops\data\raw
📁 Reports: d:\Ingénierie3\MLops\reports\deepchecks

📦 Chargement exclusif des données BRUTES crime
--------------------------------------------------------------------------------
✅ Données brutes chargées: crime_Data_2020.csv
   Shape: (199840, 28)
   Colonnes: 28
   Période: 2020-01-01 à 2020-12-31

📋 Aperçu des données brutes:


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,2020-03-01,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,2020-02-08,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,2020-11-04,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,2020-03-10,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,2020-09-09,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130



📊 Info des colonnes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199840 entries, 0 to 199839
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           199840 non-null  int64  
 1   Date Rptd       199840 non-null  object 
 2   DATE OCC        199840 non-null  object 
 3   TIME OCC        199840 non-null  int64  
 4   AREA            199840 non-null  int64  
 5   AREA NAME       199840 non-null  object 
 6   Rpt Dist No     199840 non-null  int64  
 7   Part 1-2        199840 non-null  int64  
 8   Crm Cd          199840 non-null  int64  
 9   Crm Cd Desc     199840 non-null  object 
 10  Mocodes         173083 non-null  object 
 11  Vict Age        199840 non-null  int64  
 12  Vict Sex        174354 non-null  object 
 13  Vict Descent    174351 non-null  object 
 14  Premis Cd       199838 non-null  float64
 15  Premis Desc     199771 non-null  object 
 16  Weapon Used Cd  72976 non-null   f


📊 AFFICHAGE DU RAPPORT DIRECTEMENT DANS LE NOTEBOOK :



Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_I9CBDL1HKU31QM5NYDXYJP3Z8">Data Integrity Sui…


✅ Rapport HTML sauvegardé : crime_raw_data_integrity_report.html
   Ouvre-le dans ton navigateur si tu veux une version fichier séparée.

🚨 VÉRIFICATIONS CRITIQUES (DONNÉES BRUTES)
--------------------------------------------------------------------------------
✅ Coordonnées GPS: 791 invalides (0.4%) - OK
✅ Dates: 0 invalides (0.0%) - OK
✅ Doublons: 0 (0.0%) - Acceptable

⚠️ 1 PROBLÈME(S) CRITIQUE(S) :
   🟡 ATTENTION: 41 âges aberrants

✅ NIVEAU 1 : INTÉGRITÉ DES DONNÉES BRUTES - TERMINÉ

📊 Résumé Global (données brutes):
   Lignes: 199,840 | Colonnes: 28
   Qualité globale: ⚠️ PROBLÈMES À CORRIGER
   GPS valides estimés: 99.6%
   Types de crimes distincts: 129
   Crime le plus fréquent: VEHICLE - STOLEN

💡 Prochaines étapes:
   1. 🔧 Nettoie surtout les 41 âges aberrants (valeurs négatives ou >120)

📋 Rapport interactif visible ci-dessus dans le notebook !
   Fichier HTML aussi disponible ici : d:\Ingénierie3\MLops\reports\deepchecks\crime_raw_data_integrity_report.html

